In [2]:
from __future__ import division
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.integrate import odeint
from tqdm import tqdm
import scipy.stats as st
import pandas as pd


Bad key text.latex.preview in file /home/pedroc/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /home/pedroc/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file /home/pedroc/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 418 ('savefig.jpeg_quality: 95       # when a jpeg is saved, the default qualit

In [3]:
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

In [10]:
N_boids = 1000
L = 10
delta_t = 0.1
v = np.random.uniform(0.2, 1.2, N_boids)
R = 1

x = np.random.uniform(0, L, N_boids)
y = np.random.uniform(0, L, N_boids)
theta = np.random.uniform(0, 2*np.pi, N_boids)

In [5]:
def heaviside(x):
    if x > 0:
        return 1
    else:
        return 0

In [17]:
def reorient_boid_alone(x0, y0, theta0, rand, v0, delta_t):
    new_theta = theta0 + np.random.uniform(-rand, rand)
    new_v = v0 + np.random.uniform(-v0/5, v0/5)
    
    x_new = x0 + v0*delta_t*np.cos(new_theta)
    y_new = y0 + v0*delta_t*np.sin(new_theta)
    
    return x_new, y_new, new_theta, new_v

In [12]:
def reorient_boid(x0, y0, x, y, theta, rand, v0, v, delta_t, R):
    thetas_around = theta[np.where((np.abs(x - x0) <= R) & (np.abs(y - y0) <= R))[0]]
    thetas_around_right = theta[np.where((np.abs(x+L - x0) <= R) & (np.abs(y - y0) <= R))[0]]
    thetas_around_left = theta[np.where((np.abs(x-L - x0) <= R) & (np.abs(y - y0) <= R))[0]]
    thetas_around_top = theta[np.where((np.abs(x - x0) <= R) & (np.abs(y+L - y0) <= R))[0]]
    thetas_around_bottom = theta[np.where((np.abs(x - x0) <= R) & (np.abs(y-L - y0) <= R))[0]]
    thetas_around = np.concatenate((thetas_around, thetas_around_right, thetas_around_left,
                                    thetas_around_top, thetas_around_bottom))
    
    velocities_around = v[np.where((np.abs(x - x0) <= R) & (np.abs(y - y0) <= R))[0]]
    velocities_around_right = v[np.where((np.abs(x+L - x0) <= R) & (np.abs(y - y0) <= R))[0]]
    velocities_around_left = v[np.where((np.abs(x-L - x0) <= R) & (np.abs(y - y0) <= R))[0]]
    velocities_around_top = v[np.where((np.abs(x - x0) <= R) & (np.abs(y+L - y0) <= R))[0]]
    velocities_around_bottom = v[np.where((np.abs(x - x0) <= R) & (np.abs(y-L - y0) <= R))[0]]
    velocities_around = np.concatenate((velocities_around, velocities_around_right, velocities_around_left,
                                    velocities_around_top, velocities_around_bottom))
    
    mean_sin = np.mean(np.sin(thetas_around))
    mean_cos = np.mean(np.cos(thetas_around))
    v_mean = np.mean(velocities_around)
    
    new_theta = np.arctan(mean_sin/mean_cos) + np.pi*heaviside(-mean_cos) + np.random.uniform(-rand, rand)
    new_v = (v0 - v_mean)*np.exp(-delta_t) + v_mean
    
    x_new = x0 + new_v*delta_t*np.cos(new_theta)
    y_new = y0 + new_v*delta_t*np.sin(new_theta)
    
    return x_new, y_new, new_theta, new_v

In [13]:
reorient_boid(x[0], y[0], x, y, theta, 0.1, v[0], v, delta_t, R)

(5.185721377623708, 3.8627737649694693, 4.074170307020098, 0.8710576167561741)

In [20]:
N_boids = 200
L = 10
delta_t = 0.1
max_it = 400
R = 1
t_c = [200, 201, 202]

x = np.random.uniform(0, L, N_boids)
y = np.random.uniform(0, L, N_boids)
theta = np.random.uniform(0, 2*np.pi, N_boids)
v = np.random.uniform(0.3, 1.2, N_boids)

for i in tqdm(range(max_it)):
    if i % 2 == 0:
        fig, ax = plt.subplots(figsize=(6,6))
        plt.scatter(x, y, s = 10, color = 'crimson', edgecolor = 'black')
        # plt.scatter(x[50], y[50], s = 20, color = 'teal', edgecolor = 'black')
        # circle = plt.Circle((x[50], y[50]), R, edgecolor = 'black', facecolor = 'none')
        # ax.add_patch(circle)
        plt.quiver(x, y, v*np.cos(theta), v*np.sin(theta), color = 'grey')
        plt.xlim(0, L)
        plt.ylim(0, L)
        plt.xticks([])
        plt.yticks([])
        if i < 100:
            plt.title('Collective behavior in flocks of birds = OFF', fontsize = 18, pad = 16)
        else:
            plt.title('Collective behavior in flocks of birds = ON', fontsize = 18, pad = 16)
        plt.savefig(f'Gifs/birds_{i}.png', dpi = 300, bbox_inches = 'tight')
        plt.close()
    x_new = x.copy()
    y_new = y.copy()
    theta_new = theta.copy()
    v_new = v.copy()
    
    if i < 100:
        for j in range(N_boids):
            x_new[j], y_new[j], theta_new[j], v_new[j] = reorient_boid_alone(x[j], y[j], theta[j], np.pi/10, v[j], delta_t)
            # if i in t_c and j == 50:
            #     theta_new[j] = theta_new[j] + np.pi/2

            if x_new[j] > L:
                x_new[j] = x_new[j] - L
            elif x_new[j] < 0:
                x_new[j] = x_new[j] + L

            if y_new[j] > L:
                y_new[j] = y_new[j] - L
            elif y_new[j] < 0:
                y_new[j] = y_new[j] + L

        x = x_new
        y = y_new
        theta = theta_new
        v = v_new
    else:
        for j in range(N_boids):
            x_new[j], y_new[j], theta_new[j], v_new[j] = reorient_boid(x[j], y[j], x, y, theta, np.pi/10, v[j], v, delta_t, R)
            # if i in t_c and j == 50:
            #     theta_new[j] = theta_new[j] + np.pi/2

            if x_new[j] > L:
                x_new[j] = x_new[j] - L
            elif x_new[j] < 0:
                x_new[j] = x_new[j] + L

            if y_new[j] > L:
                y_new[j] = y_new[j] - L
            elif y_new[j] < 0:
                y_new[j] = y_new[j] + L

        x = x_new
        y = y_new
        theta = theta_new
        v = v_new

100%|██████████| 400/400 [00:52<00:00,  7.55it/s]
